In [ ]:
import json

# === Cargar JSON ===
with open("data/simplified_chords.json", "r") as f:
    data = json.load(f)

# === Configuración ===
MAX_SEQ_LEN = 4  # Tamaño del contexto para predecir el siguiente acorde
DEFAULT_TONALITY = "<C_MAJOR>"  # Puedes cambiarlo o inferirlo más adelante

# === Construcción de datos para entrenamiento ===
training_data = []

for artist, songs in data.items():
    for song, chords in songs.items():
        # Normalizar acordes: mayúsculas, sin espacios
        normalized_chords = [chord.strip().capitalize() for chord in chords if chord.strip()]
        
        # Saltar canciones muy cortas
        if len(normalized_chords) <= MAX_SEQ_LEN:
            continue

        # Extraer pares (input_seq, target)
        for i in range(MAX_SEQ_LEN, len(normalized_chords)):
            input_seq = normalized_chords[i - MAX_SEQ_LEN:i]
            target = normalized_chords[i]
            input_with_tonality = [DEFAULT_TONALITY] + input_seq
            training_data.append((input_with_tonality, target))

# === Guardar datos procesados ===
with open("data/processed_training_data.json", "w") as f:
    json.dump(training_data, f, indent=2)

print(f"✅ Generados {len(training_data)} pares secuencia → acorde.")


Configuració global

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# === Hiperparámetros ===
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 1e-5

Dades amb les que treballarem

In [ ]:
# === Cargar datos procesados ===
with open("data/processed_training_data.json") as f:
    raw_data = json.load(f)

# === Crear vocabulario ===
token_set = set()
for seq, target in raw_data:
    token_set.update(seq)
    token_set.add(target)

token2idx = {token: i for i, token in enumerate(sorted(token_set))}
idx2token = {i: token for token, i in token2idx.items()}

with open("chord_vocab.json", "w") as f:
    json.dump(token2idx, f)

with open("data/chord_vocab.json", "r") as f:
    vocab = json.load(f)

In [ ]:
# === Dataset personalizado ===
class ChordDataset(Dataset):
    def __init__(self, data, token2idx):
        self.data = data
        self.token2idx = token2idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq, target = self.data[idx]
        seq_ids = [self.token2idx[token] for token in seq]
        target_id = self.token2idx[target]
        return torch.tensor(seq_ids), torch.tensor(target_id)

# === Modelo secuencial ===
class ChordLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out[:, -1, :])
        return output

# === Preparar DataLoader ===
dataset = ChordDataset(raw_data, token2idx)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# === Entrenar en CUDA si está disponible ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChordLSTM(len(token2idx), EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()


Train loop

In [ ]:
# === Entrenamiento ===
for epoch in range(EPOCHS):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        log_probs = torch.log_softmax(outputs, dim=-1)
        loss = nn.NLLLoss()(log_probs.view(-1, len(vocab)), targets.view(-1))
        # loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS} - Loss: {total_loss:.4f}")

In [ ]:
# === Guardar modelo  ===
torch.save(model.state_dict(), "chord_model.pth")

print("✅ Modelo entrenado y guardado correctamente.")

Test loop

In [ ]:
import torch
import torch.nn.functional as F

def generate_progression(
    model,
    token2idx,
    idx2token,
    key="<C_MAJOR>",
    structure=[("Verse", 4), ("Verse", 4), ("Bridge", 2), ("Chorus", 4)],
    sequence_length=5,
    temperature=1.0,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    model.eval()
    progression = {}

    # Semilla inicial
    current_sequence = [key] * sequence_length

    for section, length in structure:
        chords = []
        for _ in range(length):
            input_idxs = [token2idx[chord] for chord in current_sequence[-sequence_length:]]
            input_tensor = torch.tensor([input_idxs], dtype=torch.long).to(device)

            with torch.no_grad():
                output = model(input_tensor)  # [1, vocab_size]
                logits = output[0] / temperature
                probs = F.softmax(logits, dim=-1)
                next_idx = torch.multinomial(probs, num_samples=1).item()

            next_chord = idx2token[next_idx]
            chords.append(next_chord)
            current_sequence.append(next_chord)

            progression[section] = chords

    return progression

Generar cançó

In [30]:
progression = generate_progression(
    model,
    token2idx,
    idx2token,
    key="<C_MAJOR>",
    structure=[("Verse", 4), ("Verse", 4), ("Bridge", 2), ("Chorus", 4)],
    temperature=1.0
)

print("🎶 Progresión generada:")
for section, chords in progression.items():
    print(f"{section}: {chords}")


🎶 Progresión generada:
Verse: ['D', 'G#', 'C', 'Minor c']
Bridge: ['E', 'Bb']
Chorus: ['Minor b', 'G', 'Minor d', 'G']
